In [ ]:
import pandas as pd
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/CS4248_Project/raw_data

/content/drive/My Drive/CS4248_Project/raw_data


In [ ]:
train = pd.read_csv("fulltrain.csv",header=None)
test = pd.read_csv("balancedtest.csv",header=None)

## 1. Obtain the basic features

In [ ]:
#Check the first 10 lines
train.head(10)

,0,1
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...
5,1,"At a cafeteria-table press conference Monday, ..."
6,1,Stunned shock and dismay were just a few of th...
7,1,"Speaking with reporters before a game Monday, ..."
8,1,Sports journalists and television crews were p...
9,1,"SALEM, VAF;or the eighth straight world-histor..."


In [ ]:
#Change columns name
train.columns = ['Verdict','Text']

In [ ]:
#Check the first 10 lines
test.head(10)

,0,1
0,1,When so many actors seem content to churn out ...
1,1,In what football insiders are calling an unex...
2,1,In a freak accident following Game 3 of the N....
3,1,North Koreas official news agency announced to...
4,1,The former Alaska Governor Sarah Palin would b...
5,1,With the first Presidential debate just two da...
6,1,"There are fans, and then there are super-fans...."
7,1,"With its landmark decisions this week, the Uni..."
8,1,Koch Industries is defending its acquisition o...
9,1,Republican lawmakers asked increasingly tough ...


In [ ]:
#Change columns name
test.columns = ['Verdict','Text']

In [ ]:
#Word count
train['word_count'] = train['Text'].apply(lambda x: len(str(x).split(" ")))
train[['Text', 'word_count']].head()

,Text,word_count
0,"A little less than a decade ago, hockey fans w...",147
1,The writers of the HBO series The Sopranos too...,123
2,Despite claims from the TV news outlet to offe...,706
3,After receiving 'subpar' service and experienc...,706
4,After watching his beloved Seattle Mariners pr...,174


In [ ]:
#Char count
train['char_count'] = train['Text'].str.len()
train[['Text','char_count']].head()

,Text,char_count
0,"A little less than a decade ago, hockey fans w...",873
1,The writers of the HBO series The Sopranos too...,715
2,Despite claims from the TV news outlet to offe...,4443
3,After receiving 'subpar' service and experienc...,3913
4,After watching his beloved Seattle Mariners pr...,1058


In [ ]:
#Average word length

def avg_word(sentence):
    words=sentence.split()
    return (sum(len(word) for word in words)/len(words))

train['avg_word'] = train['Text'].apply(lambda x:avg_word(x))
train[['Text','avg_word']].head()

,Text,avg_word
0,"A little less than a decade ago, hockey fans w...",4.979452
1,The writers of the HBO series The Sopranos too...,4.860656
2,Despite claims from the TV news outlet to offe...,5.302128
3,After receiving 'subpar' service and experienc...,4.550355
4,After watching his beloved Seattle Mariners pr...,5.115607


In [ ]:
# The number of stop words
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop=stopwords.words('english')
train['stopwords']=train['Text'].apply(lambda sen:len([x for x in sen.split() if x in stop]))
train[['Text','stopwords']].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Text,stopwords
0,"A little less than a decade ago, hockey fans w...",46
1,The writers of the HBO series The Sopranos too...,43
2,Despite claims from the TV news outlet to offe...,219
3,After receiving 'subpar' service and experienc...,299
4,After watching his beloved Seattle Mariners pr...,59


In [ ]:
#The number of special chars
train['hashtags']=train['Text'].apply(lambda sen:len([x for x in sen.split() if x.startswith("#")]))
train[['Text','hashtags']].head()

,Text,hashtags
0,"A little less than a decade ago, hockey fans w...",0
1,The writers of the HBO series The Sopranos too...,0
2,Despite claims from the TV news outlet to offe...,0
3,After receiving 'subpar' service and experienc...,0
4,After watching his beloved Seattle Mariners pr...,0


In [ ]:
#The number of numerics
train['numerics']=train['Text'].apply(lambda sen:len([x for x in sen.split() if x.isdigit()]))
train[['Text','numerics']].head()

,Text,numerics
0,"A little less than a decade ago, hockey fans w...",0
1,The writers of the HBO series The Sopranos too...,1
2,Despite claims from the TV news outlet to offe...,20
3,After receiving 'subpar' service and experienc...,5
4,After watching his beloved Seattle Mariners pr...,0


In [ ]:
#The number of upper vocab
train['upper']=train['Text'].apply(lambda sen:len([x for x in sen.split() if x.isupper()]))
train[['Text','upper']].head()

,Text,upper
0,"A little less than a decade ago, hockey fans w...",4
1,The writers of the HBO series The Sopranos too...,2
2,Despite claims from the TV news outlet to offe...,9
3,After receiving 'subpar' service and experienc...,13
4,After watching his beloved Seattle Mariners pr...,3


In [ ]:
train.head()

,Verdict,Text,word_count,char_count,avg_word,stopwords,hashtags,numerics,upper
0,1,"A little less than a decade ago, hockey fans w...",147,873,4.979452,46,0,0,4
1,1,The writers of the HBO series The Sopranos too...,123,715,4.860656,43,0,1,2
2,1,Despite claims from the TV news outlet to offe...,706,4443,5.302128,219,0,20,9
3,1,After receiving 'subpar' service and experienc...,706,3913,4.550355,299,0,5,13
4,1,After watching his beloved Seattle Mariners pr...,174,1058,5.115607,59,0,0,3


## 2. Preprocessing for Text

In [ ]:
#Delete HTML
from bs4 import BeautifulSoup
train['Text'] = train['Text'].apply(lambda x: BeautifulSoup(x,'html.parser').get_text())
test['Text'] = test['Text'].apply(lambda x: BeautifulSoup(x,'html.parser').get_text())
test['Text'].head()

0    When so many actors seem content to churn out ...
1     In what football insiders are calling an unex...
2    In a freak accident following Game 3 of the N....
3    North Koreas official news agency announced to...
4    The former Alaska Governor Sarah Palin would b...
Name: Text, dtype: object

In [ ]:
#Remove emoji
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
train['Text']=train['Text'].apply(lambda x: remove_emoji(x))
test['Text']=test['Text'].apply(lambda x: remove_emoji(x))

In [ ]:
#Transform to lower letter
train['Text'] = train['Text'].apply(lambda x: x.lower())
test['Text'] = test['Text'].apply(lambda x: x.lower())
test['Text'].head()

0    when so many actors seem content to churn out ...
1     in what football insiders are calling an unex...
2    in a freak accident following game 3 of the n....
3    north koreas official news agency announced to...
4    the former alaska governor sarah palin would b...
Name: Text, dtype: object

In [ ]:
#Remove punctuation
import re
train['Text'] = train['Text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
test['Text'] = test['Text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
test['Text'].head()

0    when so many actors seem content to churn out ...
1     in what football insiders are calling an unex...
2    in a freak accident following game 3 of the nb...
3    north koreas official news agency announced to...
4    the former alaska governor sarah palin would b...
Name: Text, dtype: object

In [ ]:
#Substitute number
import inflect
def to_digit(digit):
    i = inflect.engine()
    if digit.isdigit():
        output = i.number_to_words(digit)
    else:
        output = digit
    return output
train['Text'] = train['Text'].apply(lambda x: to_digit(x))
test['Text'] = test['Text'].apply(lambda x: to_digit(x))
test['Text'].head()

0    when so many actors seem content to churn out ...
1     in what football insiders are calling an unex...
2    in a freak accident following game 3 of the nb...
3    north koreas official news agency announced to...
4    the former alaska governor sarah palin would b...
Name: Text, dtype: object

In [ ]:
#Remove the stopwords
from nltk.corpus import stopwords
stop=stopwords.words('english')
train['Text']=train['Text'].apply(lambda sen:" ".join(x for x in sen.split() if x not in stop))
test['Text']=test['Text'].apply(lambda sen:" ".join(x for x in sen.split() if x not in stop))
test['Text'].head()

0    many actors seem content churn performances qu...
1    football insiders calling unexpectedly severe ...
2    freak accident following game 3 nba finals cle...
3    north koreas official news agency announced to...
4    former alaska governor sarah palin would bring...
Name: Text, dtype: object

In [ ]:
#Remove the frequency words
freq=pd.Series(' '.join(train['Text']).split()).value_counts()[:10]
freq=list(freq.index)
train['Text']=train['Text'].apply(lambda sen:' '.join(x for x in sen.split() if x not in freq))
test['Text']=test['Text'].apply(lambda sen:' '.join(x for x in sen.split() if x not in freq))
test['Text'].head()

0    many actors seem content churn performances qu...
1    football insiders calling unexpectedly severe ...
2    freak accident following game 3 nba finals cle...
3    north koreas official news agency announced to...
4    former alaska governor sarah palin bring muchn...
Name: Text, dtype: object

In [ ]:
# Remove the scarce word
freq = pd.Series(' '.join(train['Text']).split()).value_counts()[-10:]
freq = list(freq.index)
train['Text'] = train['Text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
test['Text'] = test['Text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
test['Text'].head()

0    many actors seem content churn performances qu...
1    football insiders calling unexpectedly severe ...
2    freak accident following game 3 nba finals cle...
3    north koreas official news agency announced to...
4    former alaska governor sarah palin bring muchn...
Name: Text, dtype: object

In [ ]:
# #Correct the spelling errors
# from textblob import TextBlob
# train['Text'].apply(lambda x: str(TextBlob(x).correct()))
# test['Text'].apply(lambda x: str(TextBlob(x).correct()))
# test['Text'].head()

In [ ]:
#Noise Removal
def text_cleaner(text):
    rules = [
        {r'>\s+': u'>'},  # remove spaces after a tag opens or closes
        {r'\s+': u' '},  # replace consecutive spaces
        {r'\s*<br\s*/?>\s*': u'\n'},  # newline after a <br>
        {r'</(div)\s*>\s*': u'\n'},  # newline after </p> and </div> and <h1/>...
        {r'</(p|h\d)\s*>\s*': u'\n\n'},  # newline after </p> and </div> and <h1/>...
        {r'<head>.*<\s*(/head|body)[^>]*>': u''},  # remove <head> to </head>
        {r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'},  # show links instead of texts
        {r'[ \t]*<[^<]*?/?>': u''},  # remove remaining tags
        {r'^\s+': u''}  # remove spaces at the beginning
    ]
    for rule in rules:
      for (k, v) in rule.items():
        regex = re.compile(k)
        text = regex.sub(v, text)
      text = text.rstrip()
    return text.lower()

train['Text']=train['Text'].apply(lambda x: text_cleaner(x))

In [ ]:
#Stemming
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
train['Text']=train['Text'].apply(lambda x:" ".join(ps.stem(word) for word in x.split()))
test['Text']=test['Text'].apply(lambda x:" ".join(ps.stem(word) for word in x.split()))
test['Text'].head()

0    mani actor seem content churn perform quick pa...
1    footbal insid call unexpectedli sever punish n...
2    freak accid follow game 3 nba final cleveland ...
3    north korea offici news agenc announc today mi...
4    former alaska governor sarah palin bring muchn...
Name: Text, dtype: object

In [ ]:
#Lemmatization
from textblob import Word
import nltk
nltk.download('wordnet')
train['Text']=train['Text'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
test['Text']=test['Text'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## 3. TF-IDF + Logistic Regression

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(train['Text']))
xtrain_tfv =  tfv.transform(train['Text']) 
# xvalid_tfv = tfv.transform(xvalid)
xtest_tfv = tfv.transform(test['Text'])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
#Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0, solver='lbfgs',max_iter=3000)
clf.fit(xtrain_tfv, train['Verdict'])
predictions = clf.predict(xtest_tfv)

In [ ]:
score = f1_score(test['Verdict'], predictions, average='macro')
print (score)

0.7271046886192852


In [ ]:
xtrain_tfv.shape

(48854, 1064211)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
score2 = precision_recall_fscore_support(test['Verdict'], predictions, average='macro')
print(score2)

(0.7491595685381429, 0.738, 0.7271046886192852, None)


In [ ]:
score3 = precision_recall_fscore_support(test['Verdict'], predictions, average='micro')
print(score3)

(0.738, 0.738, 0.738, None)


In [ ]:
score4 = precision_recall_fscore_support(test['Verdict'], predictions, average='weighted')
print(score4)

(0.7491595685381429, 0.738, 0.7271046886192853, None)
